In [1]:
import pandas as pd, numpy as np

In [2]:
from vivarium import Artifact
from get_draws.api import get_draws

In [3]:
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start==2019', f'age_start>=0.5'])

# Define constants

## Artifact values

In [32]:
wasting_exposure = art.load('risk_factor.child_wasting.exposure').reset_index()
wasting_paf = art.load('risk_factor.child_wasting.population_attributable_fraction').reset_index().drop(columns='affected_measure')
wasting_rr = art.load('risk_factor.child_wasting.relative_risk').reset_index()
wasting_rr_dd = wasting_rr.loc[wasting_rr.affected_entity=='diarrheal_diseases'].drop(columns=['affected_entity','affected_measure'])
wasting_rr_lri = wasting_rr.loc[wasting_rr.affected_entity=='lower_respiratory_infections'].drop(columns=['affected_entity','affected_measure'])
wasting_rr_measles = wasting_rr.loc[wasting_rr.affected_entity=='measles'].drop(columns=['affected_entity','affected_measure'])

prevalence_c302 = art.load('cause.diarrheal_diseases.prevalence') 
p_1 = wasting_exposure.loc[wasting_exposure.parameter=='cat1'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
p_2 = wasting_exposure.loc[wasting_exposure.parameter=='cat2'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
p_3 = wasting_exposure.loc[wasting_exposure.parameter=='cat3'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
p_4 = wasting_exposure.loc[wasting_exposure.parameter=='cat4'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')

ACMR = art.load('cause.all_causes.cause_specific_mortality_rate') 
csmr_c302 = art.load('cause.diarrheal_diseases.cause_specific_mortality_rate')
emr_c302 = art.load('cause.diarrheal_diseases.excess_mortality_rate')
csmr_pem = art.load('cause.protein_energy_malnutrition.cause_specific_mortality_rate')
emr_pem = art.load('cause.protein_energy_malnutrition.excess_mortality_rate')
csmr_lri = art.load('cause.lower_respiratory_infections.cause_specific_mortality_rate')
csmr_measles = art.load('cause.measles.cause_specific_mortality_rate')
paf_wasting_lri = wasting_paf.loc[wasting_paf.affected_entity=='lower_respiratory_infections'].set_index([c for c in wasting_paf.columns if 'draw' not in c and c != 'affected_entity']).drop(columns='affected_entity')
paf_wasting_measles = wasting_paf.loc[wasting_paf.affected_entity=='measles'].set_index([c for c in wasting_paf.columns if 'draw' not in c and c != 'affected_entity']).drop(columns='affected_entity')
paf_wasting_c302 = wasting_paf.loc[wasting_paf.affected_entity=='diarrheal_diseases'].set_index([c for c in wasting_paf.columns if 'draw' not in c and c != 'affected_entity']).drop(columns='affected_entity')
incidence_c302 = art.load('cause.diarrheal_diseases.incidence_rate') 
remission_c302 = art.load('cause.diarrheal_diseases.remission_rate') 
RR_lri_1 = wasting_rr_lri.loc[wasting_rr_lri.parameter=='cat1'].set_index([c for c in wasting_rr_lri.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_lri_2 = wasting_rr_lri.loc[wasting_rr_lri.parameter=='cat2'].set_index([c for c in wasting_rr_lri.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_lri_3 = wasting_rr_lri.loc[wasting_rr_lri.parameter=='cat3'].set_index([c for c in wasting_rr_lri.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_measles_1 = wasting_rr_measles.loc[wasting_rr_measles.parameter=='cat1'].set_index([c for c in wasting_rr_measles.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_measles_2 = wasting_rr_measles.loc[wasting_rr_measles.parameter=='cat2'].set_index([c for c in wasting_rr_measles.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_measles_3 = wasting_rr_measles.loc[wasting_rr_measles.parameter=='cat3'].set_index([c for c in wasting_rr_measles.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_1 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat1'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_2 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat2'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_3 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat3'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_4 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat4'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')

In [33]:
# calculate custom prevalence to be consistent
prevalence_c302 = incidence_c302 * (4.3/365)

In [36]:
remission_c302 = remission_c302 * prevalence_c302
remission_c302

draw_0    draw_1    draw_2  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.299195  2.261389  2.145357   
       1.0       2.0     2019       2020      1.246651  1.410255  1.554659   
       2.0       5.0     2019       2020      1.246651  1.410255  1.554659   
Male   0.5       1.0     2019       2020      2.341879  2.165366  2.107153   
       1.0       2.0     2019       2020      1.295071  1.482335  1.585667   
       2.0       5.0     2019       2020      1.295071  1.482335  1.585667   

                                                draw_3    draw_4    draw_5  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.332660  2.590500  1.944410   
       1.0       2.0     2019       2020      1.282798  1.627256  1.076714   
       2.0       5.0     2019       2020      1.282798  1.627256  1.076714   
Male   0.5       1.0     2019       2020      2.115394  2.574730  1.865565   
       1.0       2.0     2019       2020      1.161249  1.691471  1.081668   
       2.0       5.0     2019       2020      1.161249  1.691471  1.081668   

                                                draw_6    draw_7    draw_8  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      1.860498  2.530103  2.698929   
       1.0       2.0     2019       2020      1.271906  1.754963  1.289647   
       2.0       5.0     2019       2020      1.271906  1.754963  1.289647   
Male   0.5       1.0     2019       2020      1.810832  2.511037  2.851841   
       1.0       2.0     2019       2020      1.246731  1.623305  1.348165   
       2.0       5.0     2019       2020      1.246731  1.623305  1.348165   

                                                draw_9  ...  draw_990  \
sex    age_start age_end year_start year_end            ...             
Female 0.5       1.0     2019       2020      2.334142  ...  1.762052   
       1.0       2.0     2019       2020      1.190064  ...  1.465518   
       2.0       5.0     2019       2020      1.190064  ...  1.465518   
Male   0.5       1.0     2019       2020      2.323035  ...  1.759805   
       1.0       2.0     2019       2020      1.243044  ...  1.500630   
       2.0       5.0     2019       2020      1.243044  ...  1.500630   

                                              draw_991  draw_992  draw_993  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.025808  2.138058  2.367358   
       1.0       2.0     2019       2020      0.903783  1.262266  1.383987   
       2.0       5.0     2019       2020      0.903783  1.262266  1.383987   
Male   0.5       1.0     2019       2020      2.014283  2.194954  2.269914   
       1.0       2.0     2019       2020      0.926905  1.196304  1.395379   
       2.0       5.0     2019       2020      0.926905  1.196304  1.395379   

                                              draw_994  draw_995  draw_996  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      1.870966  1.699358  2.908970   
       1.0       2.0     2019       2020      1.113736  1.000130  1.348287   
       2.0       5.0     2019       2020      1.113736  1.000130  1.348287   
Male   0.5       1.0     2019       2020      1.864677  1.735200  2.800615   
       1.0       2.0     2019       2020      1.098239  1.151562  1.302782   
       2.0       5.0     2019       2020      1.098239  1.151562  1.302782   

                                              draw_997  draw_998  draw_999  
sex    age_start age_end year_start year_end                                
Female 0.5       1.0     2019       2020      2.536580  1.909528  1.722454  
       1.0       2.0     2019       2020      1.171076  1.361748  1.126951  
       2.0       5.0     2019 

In [35]:
incidence_c302

draw_0    draw_1    draw_2  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.832730  2.873092  2.996564   
       1.0       2.0     2019       2020      1.444948  1.626751  1.838416   
       2.0       5.0     2019       2020      1.444948  1.626751  1.838416   
Male   0.5       1.0     2019       2020      2.861203  2.805571  2.941884   
       1.0       2.0     2019       2020      1.503897  1.709529  1.875083   
       2.0       5.0     2019       2020      1.503897  1.709529  1.875083   

                                                draw_3    draw_4    draw_5  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      3.098170  3.087093  2.536172   
       1.0       2.0     2019       2020      1.626972  1.945759  1.535853   
       2.0       5.0     2019       2020      1.626972  1.945759  1.535853   
Male   0.5       1.0     2019       2020      3.008117  3.064610  2.518989   
       1.0       2.0     2019       2020      1.528869  2.022543  1.542918   
       2.0       5.0     2019       2020      1.528869  2.022543  1.542918   

                                                draw_6    draw_7    draw_8  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.521526  3.056531  3.270019   
       1.0       2.0     2019       2020      1.503430  2.019087  1.539046   
       2.0       5.0     2019       2020      1.503430  2.019087  1.539046   
Male   0.5       1.0     2019       2020      2.506909  3.029668  3.431775   
       1.0       2.0     2019       2020      1.513575  1.867629  1.581915   
       2.0       5.0     2019       2020      1.513575  1.867629  1.581915   

                                                draw_9  ...  draw_990  \
sex    age_start age_end year_start year_end            ...             
Female 0.5       1.0     2019       2020      2.812883  ...  2.689425   
       1.0       2.0     2019       2020      1.536214  ...  1.888467   
       2.0       5.0     2019       2020      1.536214  ...  1.888467   
Male   0.5       1.0     2019       2020      2.805039  ...  2.684556   
       1.0       2.0     2019       2020      1.605330  ...  1.933651   
       2.0       5.0     2019       2020      1.605330  ...  1.933651   

                                              draw_991  draw_992  draw_993  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.447330  2.613238  2.994952   
       1.0       2.0     2019       2020      1.163270  1.803064  1.761430   
       2.0       5.0     2019       2020      1.163270  1.803064  1.761430   
Male   0.5       1.0     2019       2020      2.429864  2.615621  2.871302   
       1.0       2.0     2019       2020      1.212116  1.696115  1.782867   
       2.0       5.0     2019       2020      1.212116  1.696115  1.782867   

                                              draw_994  draw_995  draw_996  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.563659  2.613283  3.644340   
       1.0       2.0     2019       2020      1.426214  1.344372  2.012596   
       2.0       5.0     2019       2020      1.426214  1.344372  2.012596   
Male   0.5       1.0     2019       2020      2.538454  2.668392  3.545519   
       1.0       2.0     2019       2020      1.406369  1.475658  1.945014   
       2.0       5.0     2019       2020      1.406369  1.475658  1.945014   

                                              draw_997  draw_998  draw_999  
sex    age_start age_end year_start year_end                                
Female 0.5       1.0     2019       2020      2.875791  2.362733  2.605822  
       1.0       2.0     2019       2020      1.498004  1.565875  1.469757  
       2.0       5.0     2019 

In [6]:
remission_c302 = 1 / (4.3/365) * prevalence_c302
remission_c302

draw_0    draw_1    draw_2  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.832730  2.873092  2.996564   
       1.0       2.0     2019       2020      1.444948  1.626751  1.838416   
       2.0       5.0     2019       2020      1.444948  1.626751  1.838416   
Male   0.5       1.0     2019       2020      2.861203  2.805571  2.941884   
       1.0       2.0     2019       2020      1.503897  1.709529  1.875083   
       2.0       5.0     2019       2020      1.503897  1.709529  1.875083   

                                                draw_3    draw_4    draw_5  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      3.098170  3.087093  2.536172   
       1.0       2.0     2019       2020      1.626972  1.945759  1.535853   
       2.0       5.0     2019       2020      1.626972  1.945759  1.535853   
Male   0.5       1.0     2019       2020      3.008117  3.064610  2.518989   
       1.0       2.0     2019       2020      1.528869  2.022543  1.542918   
       2.0       5.0     2019       2020      1.528869  2.022543  1.542918   

                                                draw_6    draw_7    draw_8  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.521526  3.056531  3.270019   
       1.0       2.0     2019       2020      1.503430  2.019087  1.539046   
       2.0       5.0     2019       2020      1.503430  2.019087  1.539046   
Male   0.5       1.0     2019       2020      2.506909  3.029668  3.431775   
       1.0       2.0     2019       2020      1.513575  1.867629  1.581915   
       2.0       5.0     2019       2020      1.513575  1.867629  1.581915   

                                                draw_9  ...  draw_990  \
sex    age_start age_end year_start year_end            ...             
Female 0.5       1.0     2019       2020      2.812883  ...  2.689425   
       1.0       2.0     2019       2020      1.536214  ...  1.888467   
       2.0       5.0     2019       2020      1.536214  ...  1.888467   
Male   0.5       1.0     2019       2020      2.805039  ...  2.684556   
       1.0       2.0     2019       2020      1.605330  ...  1.933651   
       2.0       5.0     2019       2020      1.605330  ...  1.933651   

                                              draw_991  draw_992  draw_993  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.447330  2.613238  2.994952   
       1.0       2.0     2019       2020      1.163270  1.803064  1.761430   
       2.0       5.0     2019       2020      1.163270  1.803064  1.761430   
Male   0.5       1.0     2019       2020      2.429864  2.615621  2.871302   
       1.0       2.0     2019       2020      1.212116  1.696115  1.782867   
       2.0       5.0     2019       2020      1.212116  1.696115  1.782867   

                                              draw_994  draw_995  draw_996  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      2.563659  2.613283  3.644340   
       1.0       2.0     2019       2020      1.426214  1.344372  2.012596   
       2.0       5.0     2019       2020      1.426214  1.344372  2.012596   
Male   0.5       1.0     2019       2020      2.538454  2.668392  3.545519   
       1.0       2.0     2019       2020      1.406369  1.475658  1.945014   
       2.0       5.0     2019       2020      1.406369  1.475658  1.945014   

                                              draw_997  draw_998  draw_999  
sex    age_start age_end year_start year_end                                
Female 0.5       1.0     2019       2020      2.875791  2.362733  2.605822  
       1.0       2.0     2019       2020      1.498004  1.565875  1.469757  
       2.0       5.0     2019 

In [7]:
incidence_c302 > remission_c302

draw_0  draw_1  draw_2  draw_3  \
sex    age_start age_end year_start year_end                                   
Female 0.5       1.0     2019       2020       False   False    True   False   
       1.0       2.0     2019       2020        True    True    True    True   
       2.0       5.0     2019       2020        True    True    True    True   
Male   0.5       1.0     2019       2020       False    True   False   False   
       1.0       2.0     2019       2020        True    True   False    True   
       2.0       5.0     2019       2020        True    True   False    True   

                                              draw_4  draw_5  draw_6  draw_7  \
sex    age_start age_end year_start year_end                                   
Female 0.5       1.0     2019       2020        True   False   False   False   
       1.0       2.0     2019       2020       False   False    True   False   
       2.0       5.0     2019       2020       False   False    True   False   
Male   0.5       1.0     2019       2020       False   False   False   False   
       1.0       2.0     2019       2020       False   False   False    True   
       2.0       5.0     2019       2020       False   False   False    True   

                                              draw_8  draw_9  ...  draw_990  \
sex    age_start age_end year_start year_end                  ...             
Female 0.5       1.0     2019       2020       False   False  ...     False   
       1.0       2.0     2019       2020       False   False  ...      True   
       2.0       5.0     2019       2020       False   False  ...      True   
Male   0.5       1.0     2019       2020        True   False  ...     False   
       1.0       2.0     2019       2020       False   False  ...     False   
       2.0       5.0     2019       2020       False   False  ...     False   

                                              draw_991  draw_992  draw_993  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020         False     False     False   
       1.0       2.0     2019       2020         False      True     False   
       2.0       5.0     2019       2020         False      True     False   
Male   0.5       1.0     2019       2020          True     False     False   
       1.0       2.0     2019       2020         False     False      True   
       2.0       5.0     2019       2020         False     False      True   

                                              draw_994  draw_995  draw_996  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020          True     False     False   
       1.0       2.0     2019       2020         False     False     False   
       2.0       5.0     2019       2020         False     False     False   
Male   0.5       1.0     2019       2020          True      True     False   
       1.0       2.0     2019       2020         False      True      True   
       2.0       5.0     2019       2020         False      True      True   

                                              draw_997  draw_998  draw_999  
sex    age_start age_end year_start year_end                                
Female 0.5       1.0     2019       2020         False     False     False  
       1.0       2.0     2019       2020         False     False      True  
       2.0       5.0     2019       2020         False     False      True  
Male   0.5       1.0     2019       2020         False      True      True  
       1.0       2.0     2019       2020          True      True     False  
       2.0       5.0     2019       2020          True      True     False  

[6 rows x 1000 columns]

In [8]:
"""NOTE: the artifact values for diarrheal disease remission is greater than 
remission after re-scaling to the total population. I have adjusted the remission 
rate to be equal to the number of incident cases in the population minus the mortality 
rate for the average disease duration (6 days) to get the model to behave correctly. 
This method ignores correlation between diarrheal diseases and other causes"""

#remission_c302 = incidence_c302 - (4.3 / 365) * (ACMR - csmr_c302 + emr_c302)
#(remission_c302).apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)

'NOTE: the artifact values for diarrheal disease remission is greater than \nremission after re-scaling to the total population. I have adjusted the remission \nrate to be equal to the number of incident cases in the population minus the mortality \nrate for the average disease duration (6 days) to get the model to behave correctly. \nThis method ignores correlation between diarrheal diseases and other causes'

## Wasting transition rates

These were read-in from simulation model output rather than calculating them directly given the complexity of the calculations. This should be updated as the model is finalized. The exception is for r4 and i3 (which are currently undergoing calibration)


In [37]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.1_wasting_treatment/ciff_sam/2021_09_24_16_36_30/count_data/'
wasting_transitions = pd.read_csv(output_dir + 'wasting_transition_count.csv').drop(columns='Unnamed: 0')
wasting_transitions = (wasting_transitions
                       .loc[wasting_transitions.scenario=='baseline']
                       .groupby(['input_draw','sex','age','measure']).sum()
                       .drop(columns='year')
                       .reset_index())
wasting_person_time = pd.read_csv(output_dir + 'wasting_state_person_time.csv').drop(columns='Unnamed: 0')
wasting_person_time = (wasting_person_time
                       .loc[wasting_person_time.scenario=='baseline']
                       .groupby(['input_draw','sex','age','cause']).sum()
                       .drop(columns='year')
                       .reset_index())

In [38]:
def compute_wasting_transition_rate(transition_count_parameter, person_time_state, ages):
    rate = (((wasting_transitions.loc[wasting_transitions.measure==f'{transition_count_parameter}']
           .set_index(['input_draw','sex','age']).drop(columns='measure'))
          / (wasting_person_time.loc[wasting_person_time.cause==f'{person_time_state}']
             .set_index(['input_draw','sex','age']).drop(columns='cause')))
          .groupby(['sex','age']).mean().reset_index())
    rate['sex'] = rate.sex.str.capitalize()
    rate['year_start']=2019
    rate['year_end']=2020
    rate = rate.loc[rate.age.isin(ages)]
    rate['age_start'] = np.where(rate.age=='6-11_months',0.5,
                                np.where(rate.age=='12_to_23_months', 1.0, 2.0))
    rate['age_end'] = np.where(rate.age=='6-11_months',1.0,
                                np.where(rate.age=='12_to_23_months', 2.0, 5.0))
    for i in list(range(0,1000)):
        rate[f'draw_{i}'] = rate['value']
    return rate.drop(columns=['age','value']).set_index(['sex','age_start','age_end','year_start','year_end']).sort_index()

In [39]:
ages = ['6-11_months','12_to_23_months','2_to_4']
t1 = compute_wasting_transition_rate('severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                     'severe_acute_malnutrition', ages)
r2 = compute_wasting_transition_rate('severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                     'severe_acute_malnutrition', ages)
r3 = compute_wasting_transition_rate('moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                     'moderate_acute_malnutrition', ages)
r4_obs = compute_wasting_transition_rate('mild_child_wasting_to_susceptible_to_child_wasting_event_count', 
                                     'mild_child_wasting', ages)
i1 = compute_wasting_transition_rate('moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                     'moderate_acute_malnutrition', ages)
i2 = compute_wasting_transition_rate('mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                     'mild_child_wasting', ages)
i3_obs = compute_wasting_transition_rate('susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                     'susceptible_to_child_wasting', ages)


r4_daily = 1/50
r4 = -np.log(1 - r4_daily) * 365
ap0 = 1 - np.exp(-ACMR * 1 / 365)
m_D4_daily = 1 - np.exp(-(ACMR  
        - csmr_c302 + csmr_c302 * (1 - paf_wasting_c302)
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri)
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles)) / 365)

i3_daily = ap0 * p_4 / (p_4 / (ap0+1)) + p_3 / (ap0 + 1) * r4_daily / (p_4 / (ap0+1)) - m_D4_daily
i3 = -np.log(1 - i3_daily) * 365

<ipython-input-38-454c7b47a107>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rate[f'draw_{i}'] = rate['value']


## Prevalence ratios

In [40]:
PR_1=1.060416
PR_2=1.061946
PR_3=1.044849
PR_4=0.990530

# Solve for parameters

## State prevalence values

In [41]:
# prevalence values
p_D1 = (PR_1 * p_1 * prevalence_c302) / (PR_1 * prevalence_c302 - prevalence_c302 + 1)
p_D2 = (PR_2 * p_2 * prevalence_c302) / (PR_2 * prevalence_c302 - prevalence_c302 + 1)
p_D3 = (PR_3 * p_3 * prevalence_c302) / (PR_3 * prevalence_c302 - prevalence_c302 + 1)
p_S1 = (-p_1 * prevalence_c302 + p_1) / (PR_1 * prevalence_c302 - prevalence_c302 + 1)
p_S2 = (-p_2 * prevalence_c302 + p_2) / (PR_2 * prevalence_c302 - prevalence_c302 + 1)
p_S3 = (-p_3 * prevalence_c302 + p_3) / (PR_3 * prevalence_c302 - prevalence_c302 + 1)
p_D4 = prevalence_c302 - p_D1 - p_D2 - p_D3
p_S4 = (1 - prevalence_c302) - p_S1 - p_S2 - p_S3

In [42]:
print(p_D1.mean().mean(),
      p_D2.mean().mean(), 
      p_D3.mean().mean(),
      p_D4.mean().mean())
print((p_D1/prevalence_c302).mean().mean(),
      (p_D2/prevalence_c302).mean().mean(), 
      (p_D3/prevalence_c302).mean().mean(),
      (p_D4/prevalence_c302).mean().mean())

0.0006416260716383071 0.002197603163884727 0.005400417726652936 0.01617627610519083
0.02425019697007908 0.0876790180419497 0.22132537048223408 0.6667454145057371


In [43]:
print(p_S1.mean().mean(),
      p_S2.mean().mean(), 
      p_S3.mean().mean(),
      p_S4.mean().mean())
print((p_S1/(1-prevalence_c302)).mean().mean(),
      (p_S2/(1-prevalence_c302)).mean().mean(), 
      (p_S3/(1-prevalence_c302)).mean().mean(),
      (p_S4/(1-prevalence_c302)).mean().mean())

0.022263499323322895 0.08049506743098515 0.206656610434217 0.6661688997441081
0.02286856947658191 0.0825644788359763 0.21182522113935517 0.6827417305480867


In [44]:
for param in [p_D1,p_D2,p_D3,p_D4,p_S1,p_S2,p_S3,p_S4
             ]:
    name = [x for x in globals() if globals()[x] is param][0]
    assert param.min().min()>0, 'negative values'
    assert param.max().max()<1, 'values greater than 1'
    
assert np.all((p_D1+p_D2+p_D3+p_D4+p_S1+p_S2+p_S3+p_S4).round(5)==1), 'Prevalence parameters do not all sum to one'

## Other intermediate variables

We are making that assumption that wasting relative risks apply to 
excess mortality, NOT incidence

In [45]:
m_D1 = (ACMR - csmr_c302 + emr_c302 * (1 - paf_wasting_c302) * RR_1
        - csmr_pem + emr_pem
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri) * RR_lri_1
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles) * RR_measles_1) * p_D1

m_D2 = (ACMR - csmr_c302 + emr_c302 * (1 - paf_wasting_c302) * RR_2
        - csmr_pem + emr_pem
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri) * RR_lri_2
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles) * RR_measles_2) * p_D2

m_D3 = (ACMR - csmr_c302 + emr_c302 * (1 - paf_wasting_c302) * RR_3
        - csmr_pem
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri) * RR_lri_3
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles) * RR_measles_3) * p_D3

print(m_D1.mean().mean(),
     m_D2.mean().mean(),
     m_D3.mean().mean())
print((m_D1/p_D1).mean().mean(),
     (m_D2/p_D2).mean().mean(),
     (m_D3/p_D3).mean().mean())

0.0002915380863982537 0.00027548817007614864 0.00031054212216063737
0.3789093264178939 0.11061201288018828 0.052375756298638984


In [46]:
#di_1 = (incidence_c302 * (1 - diarrhea_paf_incidence) * RR_1_incidence) * p_S1/(1-prevalence_c302)
#di_2 = (incidence_c302 * (1 - diarrhea_paf_incidence) * RR_2_incidence) * p_S2/(1-prevalence_c302)
#di_3 = (incidence_c302 * (1 - diarrhea_paf_incidence) * RR_3_incidence) * p_S3/(1-prevalence_c302)
#di_4 = (incidence_c302 * (1 - diarrhea_paf_incidence)) * p_S4/(1-prevalence_c302)
di_1 = (incidence_c302 * p_S1/(1-prevalence_c302))
di_2 = (incidence_c302 * p_S2/(1-prevalence_c302))
di_3 = (incidence_c302 * p_S3/(1-prevalence_c302))
di_4 = (incidence_c302 * p_S4/(1-prevalence_c302))


print(di_1.mean().mean(), di_2.mean().mean(), di_3.mean().mean(), di_4.mean().mean())
print((di_1/p_S1).mean().mean(), 
      (di_2/p_S2).mean().mean(), 
      (di_3/p_S3).mean().mean(), 
      (di_4/p_S4).mean().mean())

0.051360606032450906 0.17565934019110888 0.4387309086919629 1.4067635449888685
2.1295993365009505 2.1295993365009505 2.1295993365009505 2.1295993365009505


In [47]:
dr_1 = remission_c302 / prevalence_c302 * p_D1
dr_2 = remission_c302 / prevalence_c302 * p_D2
dr_3 = remission_c302 / prevalence_c302 * p_D3
dr_4 = remission_c302 / prevalence_c302 * p_D4

print(dr_1.mean().mean(), dr_2.mean().mean(), dr_3.mean().mean(), dr_4.mean().mean())
print((dr_1/p_D1).mean().mean(), 
      (dr_2/p_D2).mean().mean(), 
      (dr_3/p_D3).mean().mean(), 
      (dr_4/p_D4).mean().mean())

0.04313513311017961 0.14764405112440004 0.36270930060010537 1.086382647495882
66.96041506225647 66.96041506225647 66.96041506225647 66.96041506225647


In [48]:
b_D1 = ACMR * p_D1
b_D2 = ACMR * p_D2
b_D3 = ACMR * p_D3

r_D1tx = t1 * p_D1 
r_D1ux = r2 * p_D1
r_D2 = r3 * p_D2
r_D3 = r4 * p_D3

In [49]:
assert np.all((m_D1 / p_D1) > (m_D2 / p_D2)), 'Mortality rate of D1 state less than D2 state'
assert np.all((m_D2 / p_D2) > (m_D3 / p_D3)), 'Mortality rate of D2 state less than D3 state'

## Incidence rates

In [50]:
i_S1 = b_D1 + di_1 - dr_1 + i1*p_2 - m_D1 - r_D1tx - r_D1ux
i_S2 = b_D1 + b_D2 + 2.0*di_1 - dr_1 - dr_2 + i2*p_3 - m_D1 - m_D2 - r_D1tx - r_D2
i_S3 = b_D1 + b_D2 + b_D3 + 2.0*di_1 + di_3 - dr_1 - dr_2 - dr_3 + i3*p_4 - m_D1 - m_D2 - m_D3 - r_D3
i_D1 = -b_D1 - di_1 + dr_1 + m_D1 + r_D1tx + r_D1ux
i_D2 = -b_D1 - b_D2 - 2.0*di_1 + dr_1 + dr_2 + m_D1 + m_D2 + r_D1tx + r_D2
i_D3 = -b_D1 - b_D2 - b_D3 - 2.0*di_1 - di_3 + dr_1 + dr_2 + dr_3 + m_D1 + m_D2 + m_D3 + r_D3

In [51]:
print(i_S1.mean().mean(), i_S2.mean().mean(), i_S3.mean().mean())
print(i_D1.mean().mean(), i_D2.mean().mean(), i_D3.mean().mean())

0.14826862974431237 0.4861755961746352 1.5010056101143898
-0.0036016121310732795 0.10449436749866464 0.05266673161045244


In [52]:
RR_i3 = (i_D3 * p_S4) / (i_S3 * p_D4)
RR_i2 = (i_D2 * p_S3) / (i_S2 * p_D3)
RR_i1 = (i_D1 * p_S2) / (i_S1 * p_D2)

In [53]:
print(RR_i3.mean().mean(), RR_i2.mean().mean(), RR_i1.mean().mean())

1.4362947847079135 8.445993493770104 -0.8850361832810869


In [54]:
assert np.all((i1 * p_2).transpose().sort_index().transpose().round(8) == (i_S1 + i_D1).round(8)), 'eq1 untrue'
assert np.all((i2 * p_3).transpose().sort_index().transpose().round(8) == (i_S2 + i_D2).round(8)), 'eq2 untrue'
assert np.all((i3 * p_4).transpose().sort_index().transpose().round(8) == (i_S3 + i_D3).round(8)), 'eq3 untrue'
assert np.all((i_D3 + di_3 + b_D3 + r_D2 + r_D1tx).round(8) 
              == (r_D3 + dr_3 + m_D3 + i_D2).round(8)), 'eq4 untrue'
assert np.all((i_D2 + di_1 + r_D1ux + b_D2).round(8)
             == (r_D2 + dr_2 + i_D1 + m_D2).round(8)), 'eq5 untrue'
assert np.all((i_D1 + di_1 + b_D1).round(8) 
             == (dr_1 + m_D1 + r_D1tx + r_D1ux).round(8)), 'eq6 untrue'

In [55]:
for param in [i_S1, i_S2, i_S3, i_D1, i_D2, i_D3
             ]:
    name = [x for x in globals() if globals()[x] is param][0]
    assert param.min().min()>0, 'negative values'

AssertionError: negative values

In [ ]:
RR_i3.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)

In [ ]:
RR_i2.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)

In [ ]:
RR_i1.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)